# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev1 toc-item"><a href="#Modules-vs-Subgraphs" data-toc-modified-id="Modules-vs-Subgraphs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modules vs Subgraphs</a></div><div class="lev2 toc-item"><a href="#Map-Subgraph-Topology-to-Module-Allegiance-Matrix" data-toc-modified-id="Map-Subgraph-Topology-to-Module-Allegiance-Matrix-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Map Subgraph Topology to Module Allegiance Matrix</a></div><div class="lev3 toc-item"><a href="#Plot-Ranked-Module-Sensitivity" data-toc-modified-id="Plot-Ranked-Module-Sensitivity-211"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Plot Ranked Module Sensitivity</a></div><div class="lev2 toc-item"><a href="#Compute-Subgraph-and-Module-Dynamics" data-toc-modified-id="Compute-Subgraph-and-Module-Dynamics-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Compute Subgraph and Module Dynamics</a></div><div class="lev2 toc-item"><a href="#Temporal-Dynamics-of-Inter/Intra-Module-Processing" data-toc-modified-id="Temporal-Dynamics-of-Inter/Intra-Module-Processing-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Temporal Dynamics of Inter/Intra Module Processing</a></div><div class="lev3 toc-item"><a href="#Plot-the-average-temporal-correlation-of-subgraphs" data-toc-modified-id="Plot-the-average-temporal-correlation-of-subgraphs-231"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Plot the average temporal correlation of subgraphs</a></div><div class="lev3 toc-item"><a href="#Plot-the-temporal-correlation-between-subgraph-types" data-toc-modified-id="Plot-the-temporal-correlation-between-subgraph-types-232"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Plot the temporal correlation between subgraph types</a></div><div class="lev3 toc-item"><a href="#Module-Reorganization-via-Subgraph-Expression-Coupling" data-toc-modified-id="Module-Reorganization-via-Subgraph-Expression-Coupling-233"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>Module Reorganization via Subgraph Expression Coupling</a></div><div class="lev2 toc-item"><a href="#Variability-of-subgraph-dynamics-across-subgraph-types" data-toc-modified-id="Variability-of-subgraph-dynamics-across-subgraph-types-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Variability of subgraph dynamics across subgraph types</a></div><div class="lev2 toc-item"><a href="#Regional-variation-in-module-flexibility-and-subgraph-coupling" data-toc-modified-id="Regional-variation-in-module-flexibility-and-subgraph-coupling-25"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Regional variation in module flexibility and subgraph coupling</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_ModData = path_PeriphData + '/e02b-DynFuncModule-Population'
path_SubData = path_PeriphData + '/e03b-DynFuncSubgraph-Population'
path_ExpData = path_PeriphData + '/e04b-CmpModuleSubgraph-Population'

for path in [path_CoreData, path_PeriphData, path_ModData, path_SubData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)     
        
if not os.path.exists('./e04b-Figures'):
    os.makedirs('./e04b-Figures')

## Generate List of Data

In [ ]:
data_dict = {'module_path': glob.glob('{}/Module_Optimization.ModAssign.*.mat'.format(path_ModData)),
             'module_alleg_path': '{}/consensus_module_allegiance.npz'.format(path_ModData),
             'subgraph_path': '{}/NMF_Optimization.consensus_subgraph.npz'.format(path_SubData),
             'cfg_path': '{}/NMF_Optimization.CfgMatr.npz'.format(path_SubData),
             'subgraph_geom_null_path': glob.glob('{}/NMF_GeomNull.subgraph_seed-*.npz'.format(path_SubData)),
             'subgraph_edge_null_path': glob.glob('{}/NMF_EdgeNull.subgraph_seed-*.npz'.format(path_SubData))}

# Modules vs Subgraphs

## Map Subgraph Topology to Module Allegiance Matrix

In [ ]:
# Load Module Allegiance data
data_mda = np.load(data_dict['module_alleg_path'], mmap_mode='r')
mda_vec = convert_adj_matr_to_cfg_matr(np.expand_dims(data_mda['module_alleg'], axis=0)).reshape(-1)

# Load Subgraph Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet'] #/ data_sub['fac_subnet'].max()

# Get Data Params
n_fac = fac_subgraph.shape[0]
n_conn = fac_subgraph.shape[1]
n_null = len(data_dict['subgraph_geom_null_path'])

# Initialize buckets
msi_real = np.nan*np.zeros(n_fac)
msi_perm_null = np.nan*np.zeros((n_fac, n_null))
msi_geom_null = np.nan*np.zeros((n_fac, n_null))
msi_edge_null = np.nan*np.zeros((n_fac, n_null))

# Correlation between Subgraph and Distance Matrix
for f_id in xrange(n_fac):
    rho, pval = stats.pearsonr(mda_vec, fac_subgraph[f_id, :])
    msi_real[f_id] = rho
    
    for n_i in xrange(n_null):
        rho, pval = stats.pearsonr(mda_vec, np.random.permutation(fac_subgraph[f_id, :]))
        msi_perm_null[f_id, n_i] = rho

# Geometric Null Correlation between Subgraph and Distance Matrix
for n_i, null_path in enumerate(data_dict['subgraph_geom_null_path']):
    data_null_sub = np.load(null_path, mmap_mode='r')
    fac_subgraph = data_null_sub['fac_subnet'] #/ data_null_sub['fac_subnet'].max()

    for f_id in xrange(n_fac):
        rho, pval = stats.pearsonr(mda_vec, fac_subgraph[f_id, :])
        msi_geom_null[f_id, n_i] = rho

# Edge Null Correlation between Subgraph and Distance Matrix
for n_i, null_path in enumerate(data_dict['subgraph_edge_null_path']):
    data_null_sub = np.load(null_path, mmap_mode='r')
    fac_subgraph = data_null_sub['fac_subnet'] #/ data_null_sub['fac_subnet'].max()

    for f_id in xrange(n_fac):
        rho, pval = stats.pearsonr(mda_vec, fac_subgraph[f_id, :])
        msi_edge_null[f_id, n_i] = rho

np.savez('{}/Module_Sensitivity.npz'.format(path_ExpData),
         msi_real=msi_real,
         msi_perm_null=msi_perm_null,
         msi_geom_null=msi_geom_null,
         msi_edge_null=msi_edge_null)

### Plot Ranked Module Sensitivity
$\text{Subgraph}_{TRUE}$ -- represents groups of temporally co-varying edge strengths with in-tact geometry and topology -- Topographic sensitivity of subgraphs driven by temporal structure of geometry and topology

$\text{Subgraph}_{RW}$ -- represents the ``noise floor`` of the temporal co-variance without geometry or topology -- Topographic sensitivity of subgraphs driven purely by temporal structure

$\text{Subgraph}_{GRW}$ -- represents the ``noise floor`` of the temporal co-variance with in-tact geometry, but no topology -- Topographic sensitivity of subgraphs driven purely by temporal structure of geometry, but no topology

In [ ]:
# Load Data
df = np.load('{}/Module_Sensitivity.npz'.format(path_ExpData))

msi_real = df['msi_real']
msi_perm_null = df['msi_perm_null']
msi_geom_null = df['msi_geom_null']
msi_edge_null = df['msi_edge_null']
msi_ord_ix = np.argsort(msi_real)

n_fac, n_null = msi_geom_null.shape

### Plot Module Sensitivity Index
plt.figure()
ax = plt.subplot(111)

# Real Topography
ax.plot(np.arange(n_fac)+1, np.sort(msi_real),
        color=[0.2, 0.2, 0.2])

alpha = 0.05
subg_intra_mod = []
subg_inter_mod = []
for ix, msi in enumerate(np.sort(msi_real)):
    if np.mean(msi_geom_null.reshape(-1) > msi) < alpha:
        color=[0.75, 0.20, 0.20]
        subg_intra_mod.append(ix)
    elif np.mean(msi_geom_null.reshape(-1) < msi) < alpha:
        color=[0.20, 0.20, 0.75] 
        subg_inter_mod.append(ix)
    else:
        color=[0.20, 0.20, 0.20]
        
    ax.scatter(ix+1, msi,
               lw=0, color=color)

# Geometric and Edge Null Topography
ax.fill_between([1, n_fac],
                y1=np.percentile(msi_geom_null.reshape(-1), 5),
                y2=np.percentile(msi_geom_null.reshape(-1), 95),
                color=[1.00, 0.25, 0.25], alpha=0.2, lw=0)

ax.fill_between([1, n_fac],
                y1=np.percentile(msi_edge_null.reshape(-1), 5),
                y2=np.percentile(msi_edge_null.reshape(-1), 95),
                color=[0.25, 0.25, 1.00], alpha=0.2, lw=0)

# Axis Settings
ax.set_xlim([0, n_fac+1])
ax.set_ylim([-0.2, 0.6])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Pearson Correlation\n(Edge Strength vs Module Allegiance)');
ax.set_xlabel('Subgraphs');

plt.savefig('./e04b-Figures/Module_Sensitivity_Distribution.svg')
plt.show()
plt.close()

np.savez('{}/Module_Sensitivity.npz'.format(path_ExpData),
         msi_real=msi_real,
         msi_ord_ix=msi_ord_ix,
         msi_perm_null=msi_perm_null,
         msi_geom_null=msi_geom_null,
         msi_edge_null=msi_edge_null,
         subg_intra_mod = subg_intra_mod,
         subg_inter_mod = subg_inter_mod)

## Compute Subgraph and Module Dynamics

In [ ]:
# Load Module Sensitivity Index
df = np.load('{}/Module_Sensitivity.npz'.format(path_ExpData))
msi_ord_ix = df['msi_ord_ix']

# Load cfg data
data_cfg = np.load(data_dict['cfg_path'], mmap_mode='r')
subj_coef_full_lbl = data_cfg['cfg_name']
subj_coef_lbl = np.array([name.split('.')[0] for name in data_cfg['cfg_name']])
subj_lbl = np.unique(subj_coef_lbl)
n_subj = len(subj_lbl)
n_win_per_subj = len(np.flatnonzero(subj_coef_lbl == subj_lbl[0]))
n_win_per_run = 20
n_run_per_subj = int(n_win_per_subj / n_win_per_run)

# Load Subgraph Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet']
fac_coef = data_sub['fac_coef'][msi_ord_ix, :]    # **Order by MSI index

# Get Data Params
n_fac = fac_subgraph.shape[0]
n_conn = fac_subgraph.shape[1]
n_node = np.int(np.ceil(np.sqrt(n_conn*2)))
n_mod_seed = len(data_dict['module_path'])
n_null = 100

# Buckets for Subgraph dynamics
subgraph_expr_enrg = np.nan*np.zeros((n_fac, n_subj, n_run_per_subj))
subgraph_expr_flex = np.nan*np.zeros((n_fac, n_subj, n_run_per_subj))
subgraph_expr_corr = np.nan*np.zeros((n_fac, n_fac, n_subj, n_run_per_subj))
subgraph_expr_corr_null = np.nan*np.zeros((n_fac, n_fac, n_subj, n_run_per_subj, n_null))

# Compute Subgraph Expression Dynamics
for s_id, subj_id in enumerate(subj_lbl):
    win_ix = np.flatnonzero(subj_coef_lbl == subj_id)
    
    run_lbl = np.unique(subj_coef_full_lbl[win_ix])
    for r_id, r_lbl in enumerate(run_lbl):
        run_ix = np.flatnonzero(subj_coef_full_lbl == r_lbl)

        sel_fac_coef = fac_coef[:, run_ix]
        subgraph_expr_corr[:, :, s_id, r_id] = np.corrcoef(sel_fac_coef)
        subgraph_expr_enrg[:, s_id, r_id] = np.mean(sel_fac_coef**2, axis=1)
        subgraph_expr_flex[:, s_id, r_id] = np.mean(np.abs(np.diff(sel_fac_coef**2, axis=1)), axis=1)

        # Null Expression
        for n_i in xrange(n_null):
            sel_fac_coef = fac_coef[:, run_ix]
            for f_id in xrange(n_fac):
                sel_fac_coef[f_id, :] = np.random.permutation(sel_fac_coef[f_id, :])
            subgraph_expr_corr_null[:, :, s_id, r_id, n_i] = np.corrcoef(sel_fac_coef)

# Compute module flexibility
module_flex = np.nan*np.zeros((n_subj, n_run_per_subj, n_node, n_mod_seed))
module_flex_node_null = np.nan*np.zeros((n_subj, n_run_per_subj, n_node, n_mod_seed, n_null))
module_flex_time_null = np.nan*np.zeros((n_subj, n_run_per_subj, n_node, n_mod_seed, n_null))
for mseed_id, mseed_path in enumerate(data_dict['module_path']):
    data_mseed = io.loadmat(mseed_path)
    print(mseed_id)
    for s_id in xrange(n_subj):
        module_assign = np.array(data_mseed['Ssubj'][0, s_id], float).T
        
        for r_id in xrange(n_run_per_subj):
            module_run = module_assign[r_id*n_win_per_run:(r_id+1)*n_win_per_run, :]
            module_flex[s_id, r_id, :, mseed_id] = np.nanmean(np.abs(np.diff(module_run, axis=0)) > 0, axis=0)
            
            # Null Flexibility
            for n_i in xrange(n_null):
                
                # Node Null
                module_run = module_assign[r_id*n_win_per_run:(r_id+1)*n_win_per_run, :]
                n_win, n_node = module_run.shape
                for iw in xrange(n_win):
                    module_run[iw, :] = np.random.permutation(module_run[iw, :])
                module_flex_node_null[s_id, r_id, :, mseed_id, n_i] = np.nanmean(np.abs(np.diff(module_run, axis=0)) > 0, axis=0)
                
                # Time Null
                module_run = module_assign[r_id*n_win_per_run:(r_id+1)*n_win_per_run, :]
                n_win, n_node = module_run.shape
                for iw in xrange(n_node):
                    module_run[:, iw] = np.random.permutation(module_run[:, iw])
                module_flex_time_null[s_id, r_id, :, mseed_id, n_i] = np.nanmean(np.abs(np.diff(module_run, axis=0)) > 0, axis=0)  
                
                
np.savez('{}/Subgraph_Module_Dynamics.npz'.format(path_ExpData),
         module_flex = module_flex,
         module_flex_node_null = module_flex_node_null,
         module_flex_time_null = module_flex_time_null,
         subgraph_expr_enrg = subgraph_expr_enrg,
         subgraph_expr_flex = subgraph_expr_flex,
         subgraph_expr_corr = subgraph_expr_corr,
         subgraph_expr_corr_null = subgraph_expr_corr_null)

## Temporal Dynamics of Inter/Intra Module Processing

In [ ]:
# Load Temporal Expression Correlation
df = np.load('{}/Subgraph_Module_Dynamics.npz'.format(path_ExpData), mmap_mode='r')
subg_expr_corr = df['subgraph_expr_corr']
subg_expr_corr_null = df['subgraph_expr_corr_null']

# Load Within-Module and Between-Module Subgraphs
df = np.load('{}/Module_Sensitivity.npz'.format(path_ExpData))
subg_intra_mod = df['subg_intra_mod']
subg_inter_mod = df['subg_inter_mod']

### Distribution of Temporal Correlation of Subgraphs
## Correlation among Inter-Module Subgraphs
triu_ix, triu_iy = np.triu_indices(len(subg_inter_mod), k=1)

# Real
expr_corr_inter_mod = subg_expr_corr[np.meshgrid(subg_inter_mod, subg_inter_mod)]
expr_corr_inter_mod_dist = expr_corr_inter_mod[triu_ix, triu_iy, :, :].mean(axis=0).mean(axis=-1)

# Null
expr_corr_inter_mod_null = subg_expr_corr_null[np.meshgrid(subg_inter_mod, subg_inter_mod)]
expr_corr_inter_mod_dist_null = expr_corr_inter_mod_null[triu_ix, triu_iy, :, :, :].mean(axis=0).mean(axis=-1).mean(axis=-1)

## Correlation among Intra-Module Subgraphs
triu_ix, triu_iy = np.triu_indices(len(subg_intra_mod), k=1)

# Real
expr_corr_intra_mod = subg_expr_corr[np.meshgrid(subg_intra_mod, subg_intra_mod)]
expr_corr_intra_mod_dist = expr_corr_intra_mod[triu_ix, triu_iy, :, :].mean(axis=0).mean(axis=-1)

# Null
expr_corr_intra_mod_null = subg_expr_corr_null[np.meshgrid(subg_intra_mod, subg_intra_mod)]
expr_corr_intra_mod_dist_null = expr_corr_intra_mod_null[triu_ix, triu_iy, :, :, :].mean(axis=0).mean(axis=-1).mean(axis=-1)

## Correlation between Intra-Module and Inter-Module Subgraphs
# Real
expr_corr_betwn_mod = subg_expr_corr[np.meshgrid(subg_inter_mod, subg_intra_mod)]
expr_corr_betwn_mod_dist = expr_corr_betwn_mod.mean(axis=0).mean(axis=0).mean(axis=-1)

# Null
expr_corr_betwn_mod_null = subg_expr_corr_null[np.meshgrid(subg_inter_mod, subg_intra_mod)]
expr_corr_betwn_mod_dist_null = expr_corr_betwn_mod_null.mean(axis=0).mean(axis=0).mean(axis=-1).mean(axis=-1)

### Plot the average temporal correlation of subgraphs

In [ ]:
avg_expr_corr = subg_expr_corr.mean(axis=-1).mean(axis=-1)
avg_expr_corr[np.diag_indices_from(avg_expr_corr)] = 0
vmin=np.min(avg_expr_corr[np.nonzero(avg_expr_corr)])
vmax=np.max(avg_expr_corr[np.nonzero(avg_expr_corr)])

plt.figure()
ax = plt.subplot(111)
mat = ax.matshow(avg_expr_corr, cmap='rainbow',
                 vmin=vmin, vmax=vmax)
plt.colorbar(mat, ax=ax)
ax.set_axis_off()

plt.savefig('./e04b-Figures/Temporal_Corr_Subgraph_Expression.svg')
plt.show()
plt.close()

### Plot the temporal correlation between subgraph types

In [ ]:
plt.figure()
ax = plt.subplot(111)
bplot = ax.boxplot([expr_corr_inter_mod_dist,
                    expr_corr_betwn_mod_dist,
                    expr_corr_intra_mod_dist], patch_artist=True);
Echobase.Plotting.fig_format.set_box_color(bplot, 'k', [[0.0, 0.375, 1.0],
                                                        [0.0, 0.375, 1.0],
                                                        [0.0, 0.375, 1.0]])
# Axis Settings
ax.set_ylim([-0.3, 0.3])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Temporal Correlation of Subgraph Expression');
ax.set_xticklabels(['Amongst\nInter-Module\nSubgraphs', 
                    'Between\nInter- and Intra-Module\nSubgraphs',
                    'Amongst\nIntra-Module\nSubgraphs']);

plt.savefig('./e04b-Figures/Temporal_Corr_Subgraph_Expression-Module_Type.svg')
plt.show()
plt.close()

print(stats.ttest_rel(expr_corr_inter_mod_dist, expr_corr_betwn_mod_dist))
print(stats.ttest_rel(expr_corr_betwn_mod_dist, expr_corr_intra_mod_dist))
print(stats.ttest_rel(expr_corr_inter_mod_dist, expr_corr_intra_mod_dist))

### Module Reorganization via Subgraph Expression Coupling

In [ ]:
# Load Temporal Expression Correlation
df = np.load('{}/Subgraph_Module_Dynamics.npz'.format(path_ExpData), mmap_mode='r')
module_flex = df['module_flex']

# Compute module flexibility for subjects
module_flex_dist = module_flex.mean(axis=-1).mean(axis=-1).mean(axis=-1)


### Inter-Module Subgraph
plt.figure()
ax = plt.subplot(111)

m, b, rho_inter, _, _ = stats.linregress(expr_corr_inter_mod_dist,
                                         module_flex_dist)
ax.scatter(expr_corr_inter_mod_dist, module_flex_dist, 
           lw=0, s=3.0, color=[0.20, 0.20, 0.75])
x = np.array([-0.05, 0.3])
y = m * x + b
ax.plot(x, y, color=[0.2, 0.2, 0.2])

# Axis Settings
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Temporal Correlation')
ax.set_ylabel('Module Flexibility');

plt.savefig('./e04b-Figures/Inter_Module_Expr_Corr-Module_Flex.svg')
plt.show()
plt.close()


### Intra-Module Subgraph
plt.figure()
ax = plt.subplot(111)

m, b, rho_intra, _, _ = stats.linregress(expr_corr_intra_mod_dist,
                                         module_flex_dist)
ax.scatter(expr_corr_intra_mod_dist, module_flex_dist, 
           lw=0, s=3.0, color=[0.75, 0.20, 0.20])
x = np.array([-0.07, -0.02])
y = m * x + b
ax.plot(x, y, color=[0.2, 0.2, 0.2])

# Axis Settings
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Temporal Correlation')
ax.set_ylabel('Module Flexibility');

plt.savefig('./e04b-Figures/Intra_Module_Expr_Corr-Module_Flex.svg')
plt.show()
plt.close()


### Inter/Intra-Module Subgraph
plt.figure()
ax = plt.subplot(111)

m, b, rho_betwn, _, _ = stats.linregress(expr_corr_betwn_mod_dist,
                                         module_flex_dist)
ax.scatter(expr_corr_betwn_mod_dist, module_flex_dist, 
           lw=0, s=3.0, color=[0.20, 0.20, 0.20])
x = np.array([-0.05, 0.05])
y = m * x + b
ax.plot(x, y, color=[0.2, 0.2, 0.2])

# Axis Settings
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Temporal Correlation')
ax.set_ylabel('Module Flexibility');

plt.savefig('./e04b-Figures/Betwn_Module_Expr_Corr-Module_Flex.svg')
plt.show()
plt.close()


### Correlation
n_null = 100000
rho_inter_null = []
rho_intra_null = []
rho_betwn_null = []
for n_i in xrange(n_null):
    rho, pv = stats.pearsonr(np.random.permutation(module_flex_dist), expr_corr_inter_mod_dist)
    rho_inter_null.append(rho)

    rho, pv = stats.pearsonr(np.random.permutation(module_flex_dist), expr_corr_intra_mod_dist)
    rho_intra_null.append(rho)

    rho, pv = stats.pearsonr(np.random.permutation(module_flex_dist), expr_corr_betwn_mod_dist)
    rho_betwn_null.append(rho)
    
if rho_inter < 0:    
    pv_inter_null = np.mean(rho_inter_null < rho_inter)
else:
    pv_inter_null = np.mean(rho_inter_null > rho_inter)        
if rho_intra < 0:    
    pv_intra_null = np.mean(rho_intra_null < rho_intra)
else:
    pv_intra_null = np.mean(rho_intra_null > rho_intra)
if rho_betwn < 0:        
    pv_betwn_null = np.mean(rho_betwn_null < rho_betwn)
else:
    pv_betwn_null = np.mean(rho_betwn_null > rho_betwn)    

print('Inter-Module Subgraphs: rho={}, pv={}'.format(rho_inter, pv_inter_null))
print('Intra-Module Subgraphs: rho={}, pv={}'.format(rho_intra, pv_intra_null))
print('Between Inter/Intra-Module Subgraphs: rho={}, pv={}'.format(rho_betwn, pv_betwn_null))

## Variability of subgraph dynamics across subgraph types

In [ ]:
# Load Temporal Expression Correlation
df = np.load('{}/Subgraph_Module_Dynamics.npz'.format(path_ExpData), mmap_mode='r')
subg_expr_flex = df['subgraph_expr_flex']
subg_expr_enrg = df['subgraph_expr_enrg']

# Load Within-Module and Between-Module Subgraphs
df = np.load('{}/Module_Sensitivity.npz'.format(path_ExpData))
subg_intra_mod = df['subg_intra_mod']
subg_inter_mod = df['subg_inter_mod']

# Compute Group Averages
subg_expr_flex_intra = subg_expr_flex.mean(axis=-1)[subg_intra_mod, :].mean(axis=0)
subg_expr_flex_inter = subg_expr_flex.mean(axis=-1)[subg_inter_mod, :].mean(axis=0)
subg_expr_enrg_intra = subg_expr_enrg.mean(axis=-1)[subg_intra_mod, :].mean(axis=0)
subg_expr_enrg_inter = subg_expr_enrg.mean(axis=-1)[subg_inter_mod, :].mean(axis=0)


### Subgraph Flexibility
plt.figure()
ax = plt.subplot(111)

# Inter-Module Subgraphs
m, b, rho_flex_inter, _, _ = stats.linregress(subg_expr_flex_inter,
                                              module_flex_dist)
ax.scatter(subg_expr_flex_inter, module_flex_dist, 
           lw=0, s=3.0, color=[0.20, 0.20, 0.75])
x = np.array([4, 14])
y = m * x + b
ax.plot(x, y, color=[0.20, 0.20, 0.75])

# Intra-Module Subgraphs
m, b, rho_flex_intra, _, _ = stats.linregress(subg_expr_flex_intra,
                                              module_flex_dist)
ax.scatter(subg_expr_flex_intra, module_flex_dist, 
           lw=0, s=3.0, color=[0.75, 0.20, 0.20])
x = np.array([4, 14])
y = m * x + b
ax.plot(x, y, color=[0.75, 0.20, 0.20])

# Axis Settings
ax.set_ylim([0.09, 0.14])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Subgraph Flexibility')
ax.set_ylabel('Module Flexibility');

#plt.savefig('./e04b-Figures/Inter_Module_Expr_Corr-Module_Flex.svg')
plt.show()
plt.close()


### Subgraph Energy
plt.figure()
ax = plt.subplot(111)

# Inter-Module Subgraphs
m, b, rho_enrg_inter, _, _ = stats.linregress(subg_expr_enrg_inter,
                                              module_flex_dist)
ax.scatter(subg_expr_enrg_inter, module_flex_dist, 
           lw=0, s=3.0, color=[0.20, 0.20, 0.75])
x = np.array([4, 14])
y = m * x + b
ax.plot(x, y, color=[0.20, 0.20, 0.75])

# Intra-Module Subgraphs
m, b, rho_enrg_intra, _, _ = stats.linregress(subg_expr_enrg_intra,
                                              module_flex_dist)
ax.scatter(subg_expr_enrg_intra, module_flex_dist, 
           lw=0, s=3.0, color=[0.75, 0.20, 0.20])
x = np.array([4, 14])
y = m * x + b
ax.plot(x, y, color=[0.75, 0.20, 0.20])

# Axis Settings
ax.set_ylim([0.09, 0.14])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Subgraph Energy')
ax.set_ylabel('Module Flexibility');

#plt.savefig('./e04b-Figures/Inter_Module_Expr_Corr-Module_Flex.svg')
plt.show()
plt.close()

### Correlation
n_null = 100000
rho_flex_inter_null = []
rho_flex_intra_null = []
rho_enrg_inter_null = []
rho_enrg_intra_null = []
for n_i in xrange(n_null):
    rho, pv = stats.pearsonr(subg_expr_flex_inter, np.random.permutation(module_flex_dist))
    rho_flex_inter_null.append(rho)

    rho, pv = stats.pearsonr(subg_expr_flex_intra, np.random.permutation(module_flex_dist))
    rho_flex_intra_null.append(rho)

    rho, pv = stats.pearsonr(subg_expr_enrg_intra, np.random.permutation(module_flex_dist))
    rho_enrg_inter_null.append(rho)

    rho, pv = stats.pearsonr(subg_expr_enrg_inter, np.random.permutation(module_flex_dist))
    rho_enrg_intra_null.append(rho)    
    
if rho_flex_inter < 0:    
    pv_flex_inter_null = np.mean(rho_flex_inter_null < rho_flex_inter)
else:
    pv_flex_inter_null = np.mean(rho_flex_inter_null > rho_flex_inter)

if rho_flex_intra < 0:
    pv_flex_intra_null = np.mean(rho_flex_intra_null < rho_flex_intra)
else:
    pv_flex_intra_null = np.mean(rho_flex_intra_null > rho_flex_intra)
    
if rho_enrg_inter < 0:    
    pv_enrg_inter_null = np.mean(rho_enrg_inter_null < rho_enrg_inter)
else:
    pv_enrg_inter_null = np.mean(rho_enrg_inter_null > rho_enrg_inter)
    
if rho_enrg_intra < 0:
    pv_enrg_intra_null = np.mean(rho_enrg_intra_null < rho_enrg_intra)    
else:
    pv_enrg_intra_null = np.mean(rho_enrg_intra_null > rho_enrg_intra)    

print('Subgraph Flex - Inter: rho={}, pv={}'.format(rho_flex_inter, pv_flex_inter_null))
print('Subgraph Flex - Intra: rho={}, pv={}'.format(rho_flex_intra, pv_flex_intra_null))
print('Subgraph Energy - Inter: rho={}, pv={}'.format(rho_enrg_inter, pv_enrg_inter_null))
print('Subgraph Energy - Intra: rho={}, pv={}'.format(rho_enrg_intra, pv_enrg_intra_null))

## Regional variation in module flexibility and subgraph coupling
Prior work shows that individual brain regions have varying degree of flexibility in shifting between functional modules.

In [ ]:
# Load Subgraph Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet']
subg_adj = np.array([convert_conn_vec_to_adj_matr(subg_conn) for subg_conn in fac_subgraph])
n_fac = fac_subgraph.shape[0]

# Load Module Flexibility and Temporal Expression Correlation
df = np.load('{}/Subgraph_Module_Dynamics.npz'.format(path_ExpData), mmap_mode='r')
module_flex = df['module_flex']
module_flex_time_null = df['module_flex_time_null']
subg_expr_corr = df['subgraph_expr_corr']
subg_expr_corr_null = df['subgraph_expr_corr_null']

# Compute regional flexibility
node_flex = module_flex.mean(axis=-1).mean(axis=1)
n_subj, n_node = node_flex.shape

# Compute node coupling
avg_subg_expr_corr = subg_expr_corr.mean(axis=0).mean(axis=-1)  # Subgraph_ID x Subject_ID
ns_fac_subg = np.sum(subg_adj, axis=1)
node_couple = np.dot(avg_subg_expr_corr,
                     ns_fac_subg)

In [ ]:
plt.figure()
ax = plt.subplot(111)
m, b, rho, pval, _ = stats.linregress(node_couple.mean(axis=0),
                                   node_flex.mean(axis=0))
ax.scatter(node_couple.mean(axis=0), node_flex.mean(axis=0),
           lw=0, s=3.0, color=[0.20, 0.20, 0.20])
x = np.array([0.6, 1.2])
y = m * x + b
ax.plot(x, y, color=[0.20, 0.20, 0.20])

# Axis Settings
ax.set_ylim([0.04, 0.20])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Node Coupling')
ax.set_ylabel('Node Flexibility');

print('Node Flexibility and Coupling: rho={}, pv={}'.format(rho, pval))

#plt.savefig('./e04b-Figures/Inter_Module_Expr_Corr-Module_Flex.svg')
plt.show()
plt.close()

In [ ]:
node_flex.shape